# Homework 5

by Maia Rosengarten <br/>
SID: 23572580<br/>
March 27, 2017<br/>
Kaggle Score:<br/>

In [2]:
import matplotlib
import numpy as np
from numpy import linalg as LA
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import math
import scipy as sp
from scipy.stats import logistic as sig

np.set_printoptions(threshold=np.nan)
from scipy import io
import sklearn
from sklearn.preprocessing import normalize
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline

# NP FNS

In [3]:
def vec(arr): # d b 1 --> (d, 1)
    return arr.reshape((arr.shape[0], 1))

In [4]:
def arr(vec): # 1 by d --> (d, )
    return vec.flatten()

# PREPROCESSING FNS

In [ ]:
def processCategoricalVars(lstIndicesOfCatFeatures):
    '''
        Replaces all categorical variables that take on n possible values with
        n new columns in the design matrix. 
            e.g. If "Weather" takes on "sunny" or "rainy" would create two new features (columns)
                in the design matrix, one 'sunny' and the other 'rainy'. If sample point's weather is
                sunny, the new 'sunny' feature would take value 1, while 'rainy' would take 0.
        Args:
            lstIndicesOfCatFeatures (int lst) - list of indices in original design matrix where features are
                categorical. Need to be updated.
    
    '''

In [ ]:
def fillMissingValuesWithAverage(designMatrix, isCat)
    '''
        Iterates over every value of every feature and replaces '?' (missing) with
        feature mean (if quantitative) and feature mode (if categorical)
        
        Args:
            designMatrix (nxd ndarray) - training data w n sample points and d features
            isSCat (bool) - boolean set to True if feature is categorical, False if quantitative
    
    '''

In [ ]:
def fillMissingValuesWithKNN(designMatrix, k):
    '''
        Iterates over every value of every feature and replaces '?' (missing) with
        value returned by K Nearest Neighbors
        
        Args:
            designMatrix (nxd ndarray) - training data w n sample points and d features
            k (int) - determines the number of neighbors that will determine point's value
    
    '''


# UTIL FNS

In [5]:
def plotCostFN(costs):
    '''
        Plots cost function as a function of number of iterations
        Args:
            costs (ndarray) - lst of costs per iteration of gradient descent
    '''
   
    iters = [i for i in range(len(costs))]
    plt.plot(iters, costs)
    plt.xscale("log")
    plt.title("Cost Per Iteration of Gradient Descent")
    plt.xlabel('numIterations')
    plt.ylabel('cost')
    plt.show()

In [6]:
def generateCsv(aryPredictions, strCsvName):
    '''
    Writes predictions of testSet to csv file
    Args:
        aryPredictions (ndarray) - (nx1)-array of predictions given size n test (or valid) set
        strCsvName (str) - name of csv file to write to
    '''
    with open(strCsvName + '.csv', 'w') as csvfile:
        writer = csv.writer(csvfile)
        for i in range(len(aryPredictions)):
            writer.writerow([i, aryPredictions[i]])
    csvfile.close()

In [ ]:
def readCSV(strFileName):
    '''
    Reads CSV file and returns data, labels tuple
    Args:
        strFileName (str) - path of csv file to read
    
    '''

In [ ]:
def writeCleanedFeaturesToCSV(cleanedDesignMatrix):
    '''
        Writes cleaned version of design matrix (filled in missing values, processed categorical variables)
        to a CSV file to speed up work (don't need to preprocess every time you run code)
        
        Args: 
            cleanedDesignMatrix  
    
    '''

In [10]:
class DecisionTree(params):
    def __init__:
        #initialize with the params
        self.root
        self.attributeBag #<--ASK ABOUT THIS
          
    def impurity(left_label_hist, right_label_hist):
        '''
            A method that takes in the result of a split: two histograms
            that count the frequencies of labels on the ”left” and ”right” side of that split.
            Calculates and outputs a scalar value representing the impurity (”badness”) of the 
            specified split on the input data (CHECK LECTURE NOTES WHAT THIS IS)  
            
            Args:
                left_label_hist (dict) - dictionary of label values to frequencies on the left side of the split
                right_label_hist (dict) - dictionary of label values to frequencies on the right side of the split
        
        '''
        
        
    def chooseSplitRule(train_data, train_labels):
        '''
            Finds the best split rule for a Node using the impurity measure and input data. 
            Exhaustively tries threshold values from the data and chooses the combination of 
            split feature and threshold with the lowest impurity value. 
           
            Args:
               train_data (ndarray) - cleaned train data 
               train_labels (ndarray) - train labels
            
            Returns (bestSplitFeatIndex, bestSplitValue)
            
            
            
            If xi is quantitative, sort points in S by feature xi; 
            remove duplicates; [duplicate values, not points]
            try splitting between each pair of consecutive values.
            [We can radix sort the values in linear time, and if n is huge we should.]
            
            Clever Bit: As you scan sorted list from left to right, 
            you can update entropy in O(1) time per point! 
            [This is important for obtaining a fast tree-building time.] <--GO OVER THIS.
            
            Need to deal with case when a child is empty (skip that split)
    
        '''
        
        for featureIndex in range(len(train_data.shape[1])):
            lstPossibleSplits = set(train_data[, featureIndex])
            numSplitsToTry = len(lstPossibleSplits)
        left_label_hist = dict((elem,0) for elem in lstPossibleSplits)
        right_label_hist = dict((elem,0) for elem in lstPossibleSplits)
        
        #THIS IS WHAT I NEED HELP WITH --> CLARIFYING HOW TO MOVE FROM LEFT TO RIGHT AND CALCULATE THE HISTOGRAMS
        
    def computeEntropy(left_dict, right_dict):
        '''
            Given left and right sets compute the weighted average entropy.
            
            Args:
                left_before, right_before, left_after, right_after
            
            Returns weighted average entropy from
            
            QUESTION: SHOULD ARGS BE DICTIONARIES OR LISTS OF CLASSES??
        
        '''
        totalLeft = sum(left_dict.values()) #assumes inputs are dictionaries
        totalRight = sum(right_dict.values()) 
        dictLeftLabelsToFreq = dict((label, 0) for label in totalLeft.keys())
        dictRightLabelsToFreq = dict((label, 0) for label in totalRight.keys())
        for label, count in left_dict.items():
            dictLeftLabelsToFreq[label] = count/totalLeft
        for label, count in right_dict.items():
            dictRightLabelsToFreq[label] = count/totalRight 
        
        leftEntropy = 0
        for label in left_dict.keys():
            prob = dictLeftLabelsToFreq[label]
            leftEntropy -= prob * np.log(prob)
            
        rightEntropy = 0
        for label in right_dict.keys():
            prob = dictRightLabelsToFreq[label]
            rightEntropy -= prob * np.log(prob)
            
        averageEntropy = (totalLeft * leftEntropy + totalRight * rightEntropy)/(totalLeft + totalRight)
        return averageEntropy
           
        
    def computeInfoGainFromSplit(left_before, right_before, left_after, right_after):
        '''
            Computes and returns the information gain from a split
            
            Args:
                left_before (dict) - dictionary mapping labels to count in left set before split
                right_before (dict) - dictionary mapping labels to count in right set before split
                left_after (dict) - dictionary mapping labels to count in left set after split
                right_after (dict) - dictionary mapping labels to count in right set after split
        
        ''' 
        entropyBeforeSplit = measureEntropy(left_before, right_before)
        entropyAfterSplit = measureEntropy(left_after, right_after)
        intInfoGain = entropyBeforeSplit - entropyAfterSplit
        return intInfoGain
        
    
    def train(train_data, train_labels, height_cap):
        '''
            Sets the root of the decision tree to the root node of the resulting tree from buildTree.
            Args:
                train_data (ndarray) - cleaned training data
                train_lablels (ndarray) - cleaned training labels
                height_cap (int) - prevents tree from growing too large
        
        '''
        
        root = buildTree(train_data, train_labels, height_cap)
        this.root = root 
    
    def buildTree(train_data, train_labels, height_cap):
        '''
            Recursively grows a decision tree by constructing nodes. Using the impurity and segmenter methods, 
            attempts to find a configuration of nodes that best splits the input data. 
            This function figures out the split rules that each node should have and figures out 
            when to stop growing the tree and insert a leaf node.  
            
            This phase is training the data.
            
            Args:
                train_data (ndarray) - n by d matrix of cleaned training data
                train_labels (ndarray) - n by 1 vector of training labels
                height_cap (int) - caps the height of the tree to prevent it from growing too large
                
        '''
        
        setSize = train_data.shape[0]
        lstUniqueLabels = set(train_labels)
        if len(lstUniqueLabels)==1:
            label = lstUniqueLabels[0]
            return Node(None, None, None, None, label)
        else:
            bestSplitFeatIndex, bestSplitVal = chooseSplitRuleAndValue(train_data, train_labels)
            left = [] #maybe should be ditionaries?
            right = [] #maybe should be dictionaries?
            for sample in range(len(train_data.shape[0])):
                if sample[bestSplitFeatIndex] <= bestSplitValue:
                    left.append(sample)
                else:
                    right.append(sample)
            return Node(bestSplitFeatIndex, bestSplitVal,  buildTree(left), buildTree(right))                     
    
    def predict(test_data):
        '''
            Given a data point, traverse the tree to find the best label to classify the data point. 
            Start at the root node and evaluate split rules as you traverse until you reach a leaf. 
            Choose that leaf nodes label as your output label.
            
            test_data (ndarray) - n by d matrix of points to classify. 
        
        '''
        
        
    
    class Node:
        def init(self, featureIndex, splitVal, rightChild, leftChild, label):
            self.splitFeatIndex = featureIndex 
            self.splitValue = splitVal
            self.left = leftChild
            self.right = rightChild 
            self.label = label #if set, this node is a leaf (mode), else is intermediary
            

SyntaxError: invalid syntax (<ipython-input-10-dcd7a6c67270>, line 2)

In [ ]:
class RandomForest(params)
    def __init__:
        self.attributeBag #subset of features to train each node on in different trees
        self.dataBag #subset of data to train each tree on 

# LOAD DATASET

In [8]:
#SPAM DATA

spamData = sp.io.loadmat("dist/spam_data.mat")
spamData.keys()
spamTrainX = spamData['training_data']
spamTrainY = spamData['training_labels']
spamTest = spamData['test_data']

spamTest = sklearn.preprocessing.normalize(spamTest.astype(float))
spamTrainX = sklearn.preprocessing.normalize(spamTrainX.astype(float))

combined = np.hstack([spamTrainX, spamTrainY])
np.random.shuffle(combined)
spamTrainX = combined[:, :-1]
spamTrainY = combined[:, -1]

#maybe add a bias feature

trainX, validX, trainY, validY = train_test_split(trainX, trainY, test_size=.1, random_state=42)
print("")

dict_keys(['__header__', '__version__', '__globals__', 'training_labels', 'test_data', 'training_data'])